### Обучение без учителя: работа с текстовыми данными, их представление. Задача кластеризации.

В этом модуле мы рассмотрим сразу несколько интересных задач:<br>
1. Способы, с помощью которых можно представлять текст для алгоритмов машинного обучения. <br>
2. Алгоритм кластеризации K-Means <br>
3. Сингулярное разложение - способ, позволяющий найти наиболее емкое и эффективное представление вашей матрицы матрицей меньшей размерности <br>
4. Соединим все задачи вместе, кластеризуя текста из Википедии, представленные с помощью алгоритма TF-IDF. <br>

Каждая задача будет привязана к упражнению, которое вам нужно будет выполнить.

In [1]:
import os
import random
import math
import copy
from collections import Counter

import nltk
from nltk.tokenize import regexp_tokenize
from sklearn.decomposition import TruncatedSVD

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from IPython.display import Image
%matplotlib inline

Зададим в качестве констант значения, которые мы будем часто использовать как гиперпараметр наших алгоритмов.

In [2]:
ITER_NUM = 10
SEED = 257

В качестве датасета мы подготовили для вас множество различных статей с Википедии, каждая из которых хранится в отдельном файле. Каждый файл содержит текст, очищенный от HTML \ CSS. 
<br>Считаем в словарь texts загруженные с Википедии данные. Ключом будет заголовок статьи, значением - текст статьи. 

In [3]:
import json

data_file = "data/unsupervised.json"
with open(data_file) as in_file:
    texts = json.load(in_file)

Посмотрим, какие страницы есть в нашем наборе данных.

In [4]:
texts.keys()

dict_keys(['Задача Плато', 'Неуверенные данные', 'Уравнение Гамильтона — Якоби — Беллмана', 'Выделение признаков', 'Субдифференциал', 'Признаковое описание', 'Максимальный разрез графа', 'Алгоритм Rete', 'Матрица мер конвергенции', 'Эволюционное моделирование', 'Двоичный поиск', 'Перекрёстная энтропия', 'Keras', 'Головоломка Конвея', 'Алгоритм Баума — Велша', 'Венгерский алгоритм', 'Марковский процесс принятия решений', 'Метод Хука — Дживса', 'Лагранжева система', 'Глубокое обучение', 'Метод k-ближайших соседей', 'Стохастическая оптимизация', 'Нейронный газ', 'Алгоритм имитации отжига', 'Иерархическая кластеризация', 'Метод Ньютона', 'М-оценки', 'Дерево решений', 'Матрица расстояний', 'Gerasim@Home', 'Матроид', 'Квадратичная задача о назначениях', 'Логистическая регрессия', 'Нечёткий регулятор', 'Метод k-средних', 'Нелинейное программирование', 'Метод штрафов', 'Стохастическое программирование', 'Геометрический центр', 'Тождества Нётер', 'Вариация отображения', 'Свёрточная нейронная се

Переведем весь текст в нижний регистр - это довольно частый прием работы с текстовыми данными.

In [5]:
for page in texts:
    texts[page] = texts[page].lower()

In [6]:
texts["Барьерная функция"]

'барьерная функция — непрерывная функция, значение которой в точке стремится к бесконечности при приближении точки к границе области допустимых решений.  барьерная функция используется в задачах оптимизации как поправочный член чтобы гарантировать наличие решений в допустимой области. например, когда ищется оптимальное значение функции     f ( x )   {\\displaystyle f(x)}  , переменная     x   {\\displaystyle x}   может быть ограничена значением, строго меньшим, чем некоторая константа     b   {\\displaystyle b}  , путём замены  функции на       f ( x ) − log \u2061 ( b − x ) .   {\\displaystyle f(x)-\\log(b-x).}   при этом функция       x ↦ − log \u2061 ( b − x )   {\\displaystyle x\\mapsto -\\log(b-x)}   играет роль барьерной функции. двумя наиболее используемыми типами барьерных функций являются обратные барьерные функции и логарифмические барьерные функции.  возобновление интереса к логарифмическим барьерным функциям вызвано их связью с двойственно-прямыми методами внутренней точки.

Как мы видим, сейчас весь текст хранится в виде одной длинной строки, в которой помимо нужного нам текста так же есть некоторая разметка текста, спецэффичная для википедии. Чтобы эффективно работать с текстами, их, во-первых, нужно отчистить от мусора, а во-вторых - провести процедуру токенизации. <br>
Токенизация - процесс автоматического выделения слов и других языковых сущностей из текста. То есть после процедуры токенизации, примененной к тексту, мы получим на выходе лист, содержащий все слова и другие языковые конструкции. 
<br><br>
Будем использоваться токенизатор, основанный на регулярных выражениях. Данное регулярное выражение ("[a-zа-я]+") указывает на то, что в тексте необходимо оставить только те слова, которые содержат русские или английские буквы.

In [7]:
tokenizer = nltk.tokenize.RegexpTokenizer(r"[a-zа-я]+")

tokens = {}
for page in texts:
    tokens[page] = tokenizer.tokenize(texts[page])

Посмотрим, что получилось в результате токенизации

In [8]:
tokens["Барьерная функция"]

['барьерная',
 'функция',
 'непрерывная',
 'функция',
 'значение',
 'которой',
 'в',
 'точке',
 'стремится',
 'к',
 'бесконечности',
 'при',
 'приближении',
 'точки',
 'к',
 'границе',
 'области',
 'допустимых',
 'решений',
 'барьерная',
 'функция',
 'используется',
 'в',
 'задачах',
 'оптимизации',
 'как',
 'поправочный',
 'член',
 'чтобы',
 'гарантировать',
 'наличие',
 'решений',
 'в',
 'допустимой',
 'области',
 'например',
 'когда',
 'ищется',
 'оптимальное',
 'значение',
 'функции',
 'f',
 'x',
 'displaystyle',
 'f',
 'x',
 'переменная',
 'x',
 'displaystyle',
 'x',
 'может',
 'быть',
 'ограничена',
 'значением',
 'строго',
 'меньшим',
 'чем',
 'некоторая',
 'константа',
 'b',
 'displaystyle',
 'b',
 'пут',
 'м',
 'замены',
 'функции',
 'на',
 'f',
 'x',
 'log',
 'b',
 'x',
 'displaystyle',
 'f',
 'x',
 'log',
 'b',
 'x',
 'при',
 'этом',
 'функция',
 'x',
 'log',
 'b',
 'x',
 'displaystyle',
 'x',
 'mapsto',
 'log',
 'b',
 'x',
 'играет',
 'роль',
 'барьерной',
 'функции',
 'дву

Приведем все слова в начальную форму, используя snowball-стеммер (он же стеммер Портера) для русского языка из библиотеки NLTK. <br>
Впервые свой алгоритм стемминга Портер предложил в 1980, после чего стеммер обрел популярность при работе с компьютерной обработкой текстов, особенно - в поисковых движках. Из-за популярности алгоритма другие разработчики стали его модифицировать и улучшать, сохраняя название. Получилось, что у алгоритма Портера много реализаций, что в какой-то мере смущало его создателя. Для исправления этого, он создал проект snowballstem (отсюда и название snowball-стеммер), ядром которого стал собственный язык программирования для удобного создания стеммеров. <br>

In [9]:
stemmer = nltk.snowball.RussianStemmer()
stemmed_tokens = {}

for page in tokens:
    stemmed_tokens[page] = []
    for token in tokens[page]:
        stemmed_tokens[page].append(stemmer.stem(token))

Посмотрим, что получилось после процедуры стемминга. Можем заметить, что теперь все слова так или иначе переведы в начальную форму.

In [10]:
stemmed_tokens["Барьерная функция"]

['барьерн',
 'функц',
 'непрерывн',
 'функц',
 'значен',
 'котор',
 'в',
 'точк',
 'стрем',
 'к',
 'бесконечн',
 'при',
 'приближен',
 'точк',
 'к',
 'границ',
 'област',
 'допустим',
 'решен',
 'барьерн',
 'функц',
 'использ',
 'в',
 'задач',
 'оптимизац',
 'как',
 'поправочн',
 'член',
 'чтоб',
 'гарантирова',
 'налич',
 'решен',
 'в',
 'допустим',
 'област',
 'например',
 'когд',
 'ищет',
 'оптимальн',
 'значен',
 'функц',
 'f',
 'x',
 'displaystyl',
 'f',
 'x',
 'перемен',
 'x',
 'displaystyl',
 'x',
 'может',
 'быт',
 'огранич',
 'значен',
 'строг',
 'меньш',
 'чем',
 'некотор',
 'констант',
 'b',
 'displaystyl',
 'b',
 'пут',
 'м',
 'зам',
 'функц',
 'на',
 'f',
 'x',
 'log',
 'b',
 'x',
 'displaystyl',
 'f',
 'x',
 'log',
 'b',
 'x',
 'при',
 'эт',
 'функц',
 'x',
 'log',
 'b',
 'x',
 'displaystyl',
 'x',
 'mapst',
 'log',
 'b',
 'x',
 'игра',
 'рол',
 'барьерн',
 'функц',
 'двум',
 'наибол',
 'используем',
 'тип',
 'барьерн',
 'функц',
 'явля',
 'обратн',
 'барьерн',
 'функц',


### Подход BoW (Bag of Words)

Для того, чтобы алгоритмы машинного обучения могли работать со словами, их необходимо представить в виде чисел, а еще лучше - векторов. Существует множество подходов, как можно представить слово или даже текст в удобном для алгоритма виде. Начнем с самого простого: "мешка слов" (BoW - Bag of Words). <br>
Идея мешка слов предельно проста. Давайте предположим, что наш текст - это вектор, каждая координата которого - это какое-то конкретное слово. Например, 1-я координата - слово "программирование", 2-я - слово "язык" и так далее. Размерность вектора - это количество слов в нашем словаре. <br>
Затем, после того как мы создали такой вектор, необходимо заполнить его какими-то значениями. В подходе с мешком слов применяется простая эвристика - давайте просто посчитаем, сколько раз каждое слово встречаеся в данном тексте. <br> 
В продолжении нашего примера: если в 1-ом тексте слово "программирование" встретилось 10 раз, то значение этой координаты будет равно 10. Если в этом же тексте слово "язык" встретилось 5 раз, то вторая координата будет равна 5. То есть получается следующий вектор: <br> [10, 5, ..., ...]
<br><br>
Давайте попроуем это реализовать!

In [11]:
bows = {}
for page in stemmed_tokens:
    bows[page] = Counter(stemmed_tokens[page])

In [12]:
token_count = Counter()
for page in bows:
    for token in bows[page]:
        token_count[token] += 1

Посмотрим на 10 самых частых слов в тексте о "Барьерной функции". Для этого вызовем функию most_common(N) (возвращает N самых частых элементов) у объекта Counter. 

In [13]:
bows["Барьерная функция"].most_common(10)

[('x', 35),
 ('b', 30),
 ('displaystyl', 20),
 ('i', 20),
 ('функц', 19),
 ('барьерн', 12),
 ('log', 12),
 ('t', 10),
 ('a', 7),
 ('в', 6)]

***ЗАДАНИЕ №1:*** найдите и отправьте в в учебную платформу самое частый токен в тексте про 'K-means++'

In [14]:
bows["K-means++"].most_common(10)

[('обучен', 13),
 ('метод', 9),
 ('центроид', 8),
 ('прав', 7),
 ('k', 7),
 ('и', 7),
 ('алгоритм', 6),
 ('сет', 6),
 ('в', 6),
 ('не', 6)]

### Подход TF-IDF (term frequency - inverse document frequency)

В подходе с мешком слов есть одна проблема: большие значение чаще всего получают различные предлоги и слова-связки. Мы же хотим выстроить такой вектор, характеризующий текст, который бы наиболее полно отражал тематику текста. Т.е. боьшие значение должны быть у той компоненты вектора, которая наиболее характерна для этого текста. <br>
Подход BoW можно довольно легко улучшить, чтобы устранить данный недостаток. Можно это сделать с помощью подхода TF-IDF. <br>
В подходе TF-IDF мы считаем каждую компоненту вектора следующим образом:<br><br>
x = TF * IDF<br>
TF - term frequency -- частота данного слова в данном тексте. Это мы уже посчитали в подходе BoW<br>
IDF - inverse document frequency -- обратная частота документа. Считаем, сколько раз слово встретилось во всех документах, и делим это число на количество документов. Затем берем логарифм (1 / получившееся на прошлом шаге число).<br>
Интуиция за этим следующая: чем чаще слово встречается во всех текстах, тем менее оно информативно, и тем меньше будет его IDF. Таким образом TF*IDF тоже будет маленьким числом.<br>
<br>

In [15]:
idf = {}

for (token, count) in token_count.most_common():
    freq = count / len(bows)
    idf[token] = math.log(1 / freq)

Посмотрим на значение idf для предлога "в". Видим, что его значение равно 0. Это связано с тем, что он встречается в каждом тексте, и поэтому вес его равен нулю - по нему невозможно определить, например, тематику текста, поэтому мы считаем его не очень важным. <br>
С другой стороны, значение idf для слова "класификатор" имеет ненулевое значение. Это говорит о том, что это слово встречается в некоторых текстах, и может быть хорошим признаком текстов на какую-то определенную тематику. 

In [16]:
print('в', idf['в'])
print('классификатор', idf['классификатор'])

в 0.0
классификатор 1.6363453653540248


Посчитаем значение TF-IDF для каждого слова в каждом тексте. Получаются они очень легко: путем перемножения TF и IDF. 

In [17]:
tfidfs = {}
for page in bows:
    tfidfs[page] = {}
    for token in bows[page]:
        tfidfs[page][token] = bows[page][token] * idf[token]

Посмотрим на слова, которые имеют самое большое значение TF-IDF для текста про "Барьерную функцию". Как можно заметить, самый высокий скор у "барьерн".

In [18]:
page = 'Барьерная функция'
for token in sorted(tfidfs[page].keys(), key = lambda t: -tfidfs[page][t]):
    print(token, tfidfs[page][token])

барьерн 53.27646895512672
log 33.37773203588433
b 26.10519149337568
логарифмическ 20.0465607455547
x 15.775259489369983
ax 9.783152249756741
i 9.769240553582645
стрем 8.490803501479377
функц 8.475952322959824
оптимизируем 8.433124389892699
displaystyl 6.90722368076918
mathbf 6.898918748292866
строг 6.565241842962193
t 5.895581445505011
поправочн 5.82600010738045
возобновлен 5.82600010738045
вдал 5.82600010738045
emptyset 5.82600010738045
cases 5.562955339314055
barrier 5.1328529268205045
бесконечн 4.9174085547879525
inft 4.84960545143659
огранич 4.65898509182794
вблиз 4.43970574626056
jorg 4.43970574626056
ucl 4.43970574626056
допустим 4.224856081352285
минус 4.216562194946349
professor 4.216562194946349
liev 4.216562194946349
noceda 4.034240638152395
mapst 3.880089958325137
барьер 3.880089958325137
wright 3.880089958325137
vandenbergh 3.880089958325137
гарантирова 3.5234150143864045
ny 3.4281048345820797
вызва 3.34109345759245
steph 3.34109345759245
зам 3.2610507499189136
смотр 3.1869

***ЗАДАНИЕ №2:*** найдите и отправьте в EdX слово с самым большим значением TF-IDF в тексте про 'K-means++'

In [20]:
page = "K-means++"
for token in sorted(tfidfs[page].keys(), key = lambda t: -tfidfs[page][t]):
    print(token, tfidfs[page][token])

центроид 32.27392510521916
обучен 14.281959752685427
rnd 14.182163456137022
means 13.712419338328319
dx 11.742513397937142
подсчитыва 8.87941149252112
apach 6.856209669164159
сет 6.753118449528204
выбра 6.6064513499392525
марковск 6.310019461324365
тех 6.264991467291246
сумм 6.0877400567051225
регламентир 5.82600010738045
вассильвитск 5.82600010738045
расстоянияэт 5.82600010738045
укажет 5.82600010738045
совпа 5.82600010738045
пор 5.381011782902601
превыс 5.1328529268205045
учител 4.954838512454439
learning 4.9235210289834415
нужн 4.819477206613031
нестабильн 4.727387818712341
артур 4.727387818712341
серге 4.727387818712341
ближайш 4.565805042528842
байесовск 4.565805042528842
точек 4.446584056580299
k 4.431302795431678
модел 4.4217884238557295
точк 4.4217884238557295
кластеризац 4.369656764740286
пок 4.083620946924379
нейрон 4.0802759661776
commons 4.038675235220261
выбор 4.012091212944932
инициализац 3.880089958325137
накоплен 3.746558565700614
суммирован 3.746558565700614
останавлив

### Применение сингулярного разложения (SVD) к матрице, полученной с помощью метода TF-IDF

SVD - один из старшейших, но при этом один из самых значимых практических результатов исследований в области линейной алгебры, который довольно часто применяется в машинном обучении. Говоря коротко, сингулярное разложение позволяет приблизить нашу исходную матрицу матрицей меньшей размерности, при этом при определенных условиях, без значимой потери информации исходной матрицы
<br><br>Для применения сингулярного разложения создадим объект DataFrame из полученной на прошлом шаге матрице, состоящей из скоров TF-IDF для каждого текста.

In [21]:
data = pd.DataFrame(tfidfs).transpose().fillna(0.0)

In [22]:
data.shape

(339, 19242)

Пусть количество скрытых компонент будет равно 20. Найдем сингулярное разложение для нашей матрицы.

In [23]:
model_svd = TruncatedSVD(n_components=20, n_iter=10, random_state=SEED)
model_svd.fit(data)

TruncatedSVD(algorithm='randomized', n_components=20, n_iter=10,
       random_state=257, tol=0.0)

In [24]:
data_transformed = model_svd.transform(data)

Теперь коилчество признаков равно 20:

In [25]:
data_transformed.shape

(339, 20)

In [26]:
def sort_by_component(svd_matrix, data, component):
    label_to_idx = {}
    for i, label in enumerate(data.index):
        label_to_idx[label] = i
    
    result = []
    for label in sorted(data.index, key = lambda k: svd_matrix[label_to_idx[k], component]):
        result.append((label, svd_matrix[label_to_idx[label], component]))
    return result 

In [27]:
sort_by_component(data_transformed, data, 0)[:10]

[('Стохастическая оптимизация', 0.5270897185449254),
 ('Гомоскедастичность', 0.5517479861380813),
 ('Перекрёстная проверка', 0.9254574112793232),
 ('Разведочный анализ данных', 0.9268141712903673),
 ('Исключение (нейронные сети)', 0.9304808699178453),
 ('Отношение рисков', 0.946913511400069),
 ('Алгоритм Эрли', 0.9658421347924238),
 ('Случайное индексирование', 1.239390579427549),
 ('Weka', 1.2632145641945498),
 ('Оптимизация развития', 1.2780618525590781)]

### Кластеризация с помощью алгоритма K-Means

Реализация алгоритма кластеризации K-Means. В данном примере мы реализуем две метрики близости: косинусная близость и евклидово расстояние. <br>
Важный момент: косинусная близость принимает значение от 0 до 1, где 1 означает максимальную близость, т.е. чем больше, тем лучше. Евклидово расстояние имеет обратное свойство - чем оно меньше, тем ближе объекты. Чтобы унифицировать данный момент мы считаем (1 - косинусное расстояние) в функции cos_dist, таким образом, чем оно меньше, тем ближе объекты. <br>

In [28]:
def get_random_centers(n_centers, points):
    centers_idx = set()
    while len(centers_idx) < n_centers:
        centers_idx.add(random.randint(0, len(points) -1))
    centers = []
    for idx in centers_idx:
        centers.append(points[idx])
    return centers

def get_clusters(centers, points, dist_func):
    result = []
    for point in points:
        cluster = 0      
        min_dist = dist_func(point, centers[0])
        for i in range(1, len(centers)):
            dist = dist_func(point, centers[i])
            if dist < min_dist:
                cluster = i
                min_dist = dist
        result.append(cluster)
    return result

def euclidean_dist(point1, point2):
    s = 0
    for dim in range(len(point1)):
        s += (point1[dim] - point2[dim]) ** 2
    return math.sqrt(s)


def cos_dist(point1, point2):
    s = 0
    s1 = 0
    s2 = 0
    for dim in range(len(point1)):
        s += point1[dim]*point2[dim]
        s1 += point1[dim] ** 2
        s2 += point2[dim] ** 2
    return 1 - (s /(math.sqrt(s1) * math.sqrt(s2)))

def get_centers(points, clusters, n_centers):
    points_count = Counter(clusters)
    centers = [np.zeros(len(points[0])) for i in range (n_centers)]
    for i in range(len(points)):
        centers[clusters[i]] += points[i]
        
    for i in range(n_centers):
        centers[i] = centers[i]/points_count[i]
    return centers
    

def kmeans(points, dist_function, n_centers, n_iterations, random_seed=31337):
    random.seed(random_seed)
    centers = get_random_centers(n_centers, points)
    for iteration in range(n_iterations):
        clusters = get_clusters(centers, points, dist_function)
        centers = get_centers(points, clusters, n_centers)
    return (centers, get_clusters(centers, points, dist_function))

In [29]:
centers, clusters = kmeans(
    data_transformed, euclidean_dist, n_centers=30, n_iterations=ITER_NUM, random_seed=SEED
)
data_with_clusters = copy.deepcopy(data)

Создадим дополнительный столбик в DataFrame data_with_clusters, который будет отвечать за принадлежность объекта к кластеру, присвоенному в результате применения k-means

In [30]:
data_with_clusters.insert(loc=0, column="#cluster", value=clusters)

Посмотрим, какие объекты находятся в разных кластерах

In [31]:
data_with_clusters[data_with_clusters["#cluster"] == 2].index

Index(['Gerasim@Home', 'Псевдолес', 'Декомпозиция графа на ветви'], dtype='object')

In [32]:
data_with_clusters[data_with_clusters["#cluster"] == 10].index

Index(['Уравнение Гамильтона — Якоби — Беллмана', 'Метод k-ближайших соседей',
       'Вариация отображения', 'Куст событий',
       'Алгоритм Бройдена — Флетчера — Гольдфарба — Шанно', 'Алгоритм Витерби',
       'Фиктивная переменная', 'Скрытая марковская модель',
       'Метод эллипсоидов', 'Минимизация эмпирического риска',
       'Автокорреляционная функция', 'Приближение с помощью кривых',
       'Задача о сумме подмножеств', 'AdaBoost', 'Нелинейная регрессия',
       'Обучение с ошибками', 'Жадный алгоритм',
       'Метод нечёткой кластеризации C-средних',
       'Фиксированные эффекты с разложением вектора', 'Алгоритм Кармаркара',
       'Байесовская вероятность'],
      dtype='object')

***Задание №3:*** Сколько элементов находится в кластере #1? Посчитайте и вставьте ответ в учебную платформу

In [35]:
data_with_clusters[data_with_clusters["#cluster"] == 1]

,#cluster,a,aaa,aaas,aach,aall,aap,aaron,ab,ababab,...,ярник,ярославск,ярушкин,ясн,ясницк,ястреб,ячеек,ячейк,яшин,ящик
Венгерский алгоритм,1,12.752554,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Двойственность (оптимизация),1,1.779426,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Симплекс-метод,1,21.649684,0.0,0.0,0.0,0.0,0.0,0.0,3.628776,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Линейное программирование,1,8.897131,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,3.746559,0.0,0.0,0.0,0.0,0.0,0.0
Регрессия (математика),1,0.296571,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Регрессионный анализ,1,1.186284,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Полуопределённое программирование,1,14.235409,0.0,0.0,0.0,0.0,0.0,0.0,18.143878,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


### Поиск под-кластеров внутри имеющихся кластеров

На прошлом шаге мы получили кластеры с помощью алгоритма K-Means. Все кластеры разных размеров, в каких-то много объектов, а в каких-то всего один. Попробуем внутри кластеров, содержащих много объектов (много в данном случае - это больше 7), поискать под-кластеры с помощью того же самого алгоритма. Это и будет ваше следующее задание (с самостоятельной оценкой)!
<br>

***Задание №4 (без проверки):*** запустите K-Means внутри получившихся кластеров, содержащих больше 7 текстов. 

### Использование K-Means из библиотеки sklearn

Так же алгоритм K-Means реализован в популярной библиотеке sklearn.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans_model = KMeans(n_clusters=30, random_state=SEED)
kmeans_model.fit(data_transformed)

In [ ]:
data_with_clusters = copy.deepcopy(data)
data_with_clusters.insert(loc=0, column="#cluster", value=kmeans_model.labels_)

In [ ]:
data_with_clusters[data_with_clusters["#cluster"] == 0].index

Видим, что в K-Means из sklearn получился кластер, очень похожий на тот, что получился в нашей реализации. Отличие связано с тем, что центры кластеров инициализируются в разных реализация по-разному, в связи с этим возникают отличия.